In [54]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as itr
import time
from boundaries import WallBoundary

# Initializing a rank three tensor filled with nodes
latticeSize = 9
xResolution = 100
yResolution = 100
relaxationTime = 0.5
# Weights
unitVect = np.array([
	[0, 0],	[1, 0], [0, 1],
	[-1, 0], [0, -1], [1, 1],
	[-1, 1], [-1, -1], [1, -1]
])
weight = np.array([
    4/9, 1/9, 1/9,
    1/9, 1/9, 1/36,
    1/36, 1/36, 1/36
])
cs = np.sqrt(3)

# Initializing the fluid matrix
fluid = np.ones((xResolution, yResolution, latticeSize)) + 0.1 * np.random.randn(xResolution, yResolution, latticeSize)
# Assigning a right velocity
fluid[:, :, 1] = 2.3

# Set a boundary
boundary = WallBoundary(xResolution, yResolution, False)
boundary.cylindricalWall([xResolution/2, yResolution/2], 13)

In [55]:
# One iteration only

fluidEquilibrium = np.ones((xResolution, yResolution, latticeSize))
iteratedFluid = np.ones((xResolution, yResolution, latticeSize))

# Internal collision step
density = np.sum(fluid, axis = 2)
mesoMomentum = fluid.reshape(xResolution, yResolution, latticeSize, 1) * unitVect
momentum = np.sum(mesoMomentum, axis = 2) / density.reshape(xResolution, yResolution, 1)
momentumMagnitude = np.linalg.norm(momentum, axis = 2) ** 2 / (2 * cs**4) # Represents u . u
momentumDot = np.zeros((xResolution, yResolution, latticeSize))

# I kinda give up on broadcasting rn. Help me :skull:
for xIndex, yIndex in itr.product(range(xResolution), range(yResolution)):
    latticeDensity = density[xIndex, yIndex]
    cellMomentumMagnitude = momentumMagnitude[xIndex, yIndex]
    for latticeIndex in range(latticeSize):
        momentumDot = np.dot(unitVect[latticeIndex], momentum[xIndex, yIndex])
        fluidEquilibrium = latticeDensity * weight[latticeIndex] * (
			1 + momentumDot / (cs ** 2) + momentumDot ** 2 / (cs ** 4) - cellMomentumMagnitude
		)
        iteratedFluid[xIndex, yIndex, latticeIndex] = fluid[xIndex, yIndex, latticeIndex] * (1 - 1/relaxationTime) + 1/relaxationTime * fluidEquilibrium